# LLM Pipeline Interactive Testing

This notebook provides an environment to interactively test and debug components of the ShuScribe LLM pipeline, including the `LLMService`, entity extraction, and wiki generation logic.

## Make sure the Portkey Gateway is running to use an LLM Service

```bash
docker run -d \
  --name portkey-gateway \
  -p 8787:8787 \
  portkeyai/gateway:latest
```

## ⚙️ Setup and Autoreload

The `%load_ext autoreload` and `%autoreload 2` magic commands ensure that any changes you make to your Python source files (`.py`) in `src/` are automatically reloaded in the notebook without needing to restart the kernel. This is crucial for rapid iteration.

We also configure basic logging for visibility.

In [ ]:
# Cell 1: Setup and Configuration for LLM Service Testing
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import asyncio
import logging
from uuid import uuid4

# Set environment to skip database for LLM service testing
os.environ["SKIP_DATABASE"] = "true"
os.environ["PORTKEY_BASE_URL"] = "http://localhost:8787/v1"  # Default Portkey Gateway

# Add the backend/src directory to sys.path so we can import our modules
# This assumes you are running the notebook from the `backend/` directory or VS Code multi-root
notebook_dir = Path.cwd()
if (notebook_dir / 'src').is_dir() and (notebook_dir / 'pyproject.toml').is_file():
    # This means we're likely in the backend/ directory itself
    sys.path.insert(0, str(notebook_dir / 'src'))
elif (notebook_dir.parent / 'src').is_dir() and (notebook_dir.parent / 'pyproject.toml').is_file():
    # This means we're likely in the backend/notebooks/ directory
    sys.path.insert(0, str(notebook_dir.parent / 'src'))
else:
    print("Warning: Could not automatically add 'src/' to Python path. Please ensure your current directory allows imports from src/")

# Configure logging for better output
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
# Reduce noise from third-party loggers
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('uvicorn.access').setLevel(logging.WARNING)
logging.getLogger('shuscribe').setLevel(logging.INFO)

print("🧪 LLM Service Testing Notebook")
print("=" * 50)
print(f"Current working directory: {os.getcwd()}")
print(f"Database mode: {'IN-MEMORY' if os.environ.get('SKIP_DATABASE') == 'true' else 'SUPABASE'}")
print(f"Portkey Gateway: {os.environ.get('PORTKEY_BASE_URL', 'Not configured')}")
print("Autoreload enabled. Changes to .py files in src/ will be reloaded.")
print("\n💡 This notebook tests LLM service functionality without requiring database setup.")
print("   Perfect for testing direct API key usage and LLM provider integrations.")

🧪 LLM Service Testing Notebook
Current working directory: /home/jimnix/gitrepos/shuscribe/backend/notebooks
Database mode: IN-MEMORY
Portkey Gateway: http://localhost:8787/v1
Autoreload enabled. Changes to .py files in src/ will be reloaded.

💡 This notebook tests LLM service functionality without requiring database setup.
   Perfect for testing direct API key usage and LLM provider integrations.


## 📦 Import Modules

Import necessary modules from your `src/` directory. This is where you'll bring in your `Settings`, `LLMService`, `UserRepository`, etc.

In [2]:
# Cell 2: Import Modules (Updated for Supabase)
from src.config import settings
from src.services.llm.llm_service import LLMService
from src.api.dependencies import get_user_repository_dependency
from src.schemas.llm.models import LLMMessage, LLMResponse
from src.core.encryption import encrypt_api_key  # Import encryption function
from dotenv import dotenv_values
from src.database.models.user import UserCreate, UserAPIKeyCreate

print("✅ Modules imported successfully.")

# Display current settings
print("\n--- Current Settings ---")
print(f"DEBUG: {settings.DEBUG}")
print(f"ENVIRONMENT: {settings.ENVIRONMENT}")
print(f"SKIP_DATABASE: {settings.SKIP_DATABASE}")
print(f"PORTKEY_BASE_URL: {settings.PORTKEY_BASE_URL}")
if not settings.SKIP_DATABASE:
    print(f"SUPABASE_URL: {settings.SUPABASE_URL}")
    print(f"SUPABASE_KEY: {'***' + settings.SUPABASE_KEY[-4:] if len(settings.SUPABASE_KEY) > 4 else 'Not set'}")
else:
    print("DATABASE_MODE: In-Memory (Supabase skipped)")
print("------------------------")

print("\n🔧 Import Summary:")
print("✅ LLMService - Ready for testing")
print("✅ Repository Factory - Automatic in-memory/Supabase switching") 
print("✅ Pydantic Models - Type-safe user schemas")
print("✅ Encryption - API key encryption/decryption")  # Updated
print("✅ Supabase Connection - Available when SKIP_DATABASE=false")

2025-06-30 20:55:46,338 - src.config - INFO - Pydantic Settings 'extra' mode set to: 'ignore' for environment: 'development'
✅ Modules imported successfully.

--- Current Settings ---
DEBUG: True
ENVIRONMENT: development
SKIP_DATABASE: True
PORTKEY_BASE_URL: http://localhost:8787/v1
DATABASE_MODE: In-Memory (Supabase skipped)
------------------------

🔧 Import Summary:
✅ LLMService - Ready for testing
✅ Repository Factory - Automatic in-memory/Supabase switching
✅ Pydantic Models - Type-safe user schemas
✅ Encryption - API key encryption/decryption
✅ Supabase Connection - Available when SKIP_DATABASE=false


## 💾 Database & Service Initialization

We need to initialize the database connection and the services. The `LLMService` requires a `UserRepository` instance, which in turn requires an `AsyncSession`. If `SKIP_DATABASE` is `True` in your `.env`, database-dependent operations will raise an error.

In [3]:
# Cell 3: Initialize LLM Service and Store Encrypted API Keys

print("🚀 Initializing LLM Service...")

# The factory automatically chooses in-memory or Supabase based on SKIP_DATABASE
llm_service = LLMService(user_repository=get_user_repository_dependency())

# Ensure repository is available
if not llm_service.user_repository:
    raise RuntimeError("Failed to initialize user repository")

print("✅ LLM Service initialized successfully!")
print(f"📁 Repository type: {type(llm_service.user_repository).__name__}")
print(f"🛡️  Database mode: {'In-Memory' if settings.SKIP_DATABASE else 'Supabase'}")

if settings.SKIP_DATABASE:
    print("\n💡 Running in database-free mode:")
    print("   • Perfect for testing with direct API keys")
    print("   • No Supabase setup required")
    print("   • User API keys stored in memory only")
else:
    print("\n🗄️  Connected to Supabase:")
    print("   • User API keys stored encrypted in database")
    print("   • Full multi-user support enabled")
    print("   • Row-level security active")

print("\n🔑 Loading and storing encrypted API keys...")

# Load environment variables once
env_values = dotenv_values()

# Create a test user for API key storage
TEST_USER = await llm_service.user_repository.create(UserCreate(
    email="test@example.com",
    display_name="Test User"
))

print(f"👤 Created test user: {TEST_USER.email} (ID: {TEST_USER.id})")

# Store API keys from environment in the repository (properly encrypted)
stored_keys = 0
available_providers = []
for provider in LLMService.get_all_llm_providers():
    provider_id = provider.provider_id
    api_key = env_values.get(f"{provider_id.upper()}_API_KEY")
    
    if api_key:
        # Properly encrypt the API key before storing
        encrypted_key = encrypt_api_key(api_key)
        
        await llm_service.user_repository.store_api_key(
            user_id=TEST_USER.id,
            api_key_data=UserAPIKeyCreate(
                provider=provider_id,
                api_key=encrypted_key,  # Now properly encrypted
                provider_metadata={}
            )
        )
        stored_keys += 1
        available_providers.append(provider.display_name)
        print(f"   🔐 Stored and encrypted {provider.display_name} API key")

print(f"✅ Stored {stored_keys} encrypted API keys in repository")
if available_providers:
    print(f"📋 Available providers: {', '.join(available_providers)}")
print("\n🎯 Ready for testing!")

🚀 Initializing LLM Service...
✅ LLM Service initialized successfully!
📁 Repository type: FileUserRepository
🛡️  Database mode: In-Memory

💡 Running in database-free mode:
   • Perfect for testing with direct API keys
   • No Supabase setup required
   • User API keys stored in memory only

🔑 Loading and storing encrypted API keys...
👤 Created test user: test@example.com (ID: 06478a58-90e4-46ed-9ca9-82cbf40bbb53)
   🔐 Stored and encrypted OpenAI API key
   🔐 Stored and encrypted Google API key
   🔐 Stored and encrypted Anthropic API key
✅ Stored 3 encrypted API keys in repository
📋 Available providers: OpenAI, Google, Anthropic

🎯 Ready for testing!


## 🔑 API Key Management Test

Test the `validate_api_key` method of the `LLMService`. You will need to provide a real API key for a supported provider (e.g., OpenAI, Anthropic, Google).

### Again, make sure the Portkey Gateway is running to use an LLM Service

```bash
docker run -d \
  --name portkey-gateway \
  -p 8787:8787 \
  portkeyai/gateway:latest
```

In [5]:
# Cell 4: API Key Validation Test (Using LLM Service with user_id)

from typing import cast

print("🔑 Testing API Key Validation via Chat Completion")
print("=" * 50)

# Ensure repository is available
if not llm_service.user_repository:
    raise RuntimeError("User repository not available")

# Test each provider that has an API key stored in the repository
providers_tested = 0
providers_passed = 0
responses = []

# Get all stored API keys for our test user to see what's available
stored_api_keys = await llm_service.user_repository.get_all_api_keys(TEST_USER.id)
print(f"📦 Found {len(stored_api_keys)} stored API keys in repository")
available_provider_ids = {key.provider for key in stored_api_keys}

for provider in LLMService.get_all_llm_providers():
    provider_id = provider.provider_id
    
    if provider_id not in available_provider_ids:
        print(f"⏭️  {provider.display_name}: SKIPPED (no API key stored)")
        continue
    
    providers_tested += 1
    print(f"🧪 Testing {provider.display_name}...")
    
    try:
        # Get the default model for this provider for testing
        test_model = LLMService.get_default_test_model_name_for_provider(provider_id)
        
        if not test_model:
            print(f"❌ {provider.display_name}: No default model configured for testing.")
            continue
            
        # Make a minimal chat completion call - LLM service will fetch API key automatically
        result = await llm_service.chat_completion(
            provider=provider_id,
            model=test_model,
            messages=[LLMMessage(role="user", content="Hello")],
            user_id=TEST_USER.id,  # LLM service will lookup API key from repository
            max_tokens=5,
            temperature=0.0,
            stream=False
        )
        result = cast(LLMResponse, result) # stream=False
        
        print(f"✅ {provider.display_name}: SUCCESS")
        print(f"   Model used: {result.model}")
        print(f"   Response: '{result.content.strip()}'")
        
        providers_passed += 1
        responses.append({
            "provider": provider.display_name,
            "model": result.model,
            "response": result.content
        })
            
    except Exception as e:
        print(f"❌ {provider.display_name}: ERROR - {e}")

print(f"\n📊 Results: {providers_passed}/{providers_tested} providers validated successfully")

if providers_tested == 0:
    print("\n💡 Tip: Run the initialization cell to store API keys from .env file")
elif providers_passed == providers_tested:
    print("🎉 All stored API keys are working!")
    
# Show actual responses
if responses:
    print(f"\n📋 Test Responses:")
    for resp in responses:
        print(f"  • {resp['provider']} ({resp['model']}): '{resp['response']}'")


🔑 Testing API Key Validation via Chat Completion
📦 Found 3 stored API keys in repository
🧪 Testing OpenAI...
2025-06-30 20:56:01,884 - src.services.llm.llm_service - INFO - Using database API key for provider=openai, model=gpt-4.1-nano, user=06478a58-90e4-46ed-9ca9-82cbf40bbb53


2025-06-30 20:56:01,949 - src.services.llm.llm_service - INFO - Making LLM request: provider=openai, model=gpt-4.1-nano, gateway=http://localhost:8787/v1, streaming=False
✅ OpenAI: SUCCESS
   Model used: gpt-4.1-nano-2025-04-14
   Response: 'Hello! How can I'
🧪 Testing Google...
2025-06-30 20:56:02,940 - src.services.llm.llm_service - INFO - Using database API key for provider=google, model=gemini-2.0-flash-001, user=06478a58-90e4-46ed-9ca9-82cbf40bbb53
2025-06-30 20:56:02,966 - src.services.llm.llm_service - INFO - Making LLM request: provider=google, model=gemini-2.0-flash-001, gateway=http://localhost:8787/v1, streaming=False
✅ Google: SUCCESS
   Model used: gemini-2.0-flash-001
   Response: 'Hello! How can I'
🧪 Testing Anthropic...
2025-06-30 20:56:03,523 - src.services.llm.llm_service - INFO - Using database API key for provider=anthropic, model=claude-3-5-haiku-latest, user=06478a58-90e4-46ed-9ca9-82cbf40bbb53
2025-06-30 20:56:03,550 - src.services.llm.llm_service - INFO - Making

In [6]:
# Cell 5: Streaming Chat Completion Test (Using LLM Service with user_id)

import asyncio
from typing import cast, AsyncIterator

print("\n⚡ Testing Streaming Chat Completion")
print("=" * 50)

# --- Configuration ---
# Change this to test a different provider (e.g., "openai", "anthropic")
TEST_PROVIDER = "google" 
# -------------------

# Ensure repository is available and check if provider has stored API key
if not llm_service.user_repository:
    raise RuntimeError("User repository not available")

stored_keys = await llm_service.user_repository.get_all_api_keys(TEST_USER.id)
available_providers = {key.provider for key in stored_keys}

if TEST_PROVIDER not in available_providers:
    print(f"⏭️  {TEST_PROVIDER.upper()}: SKIPPED (no API key stored in repository)")
    print(f"💡 Available providers: {', '.join(available_providers)}")
else:
    print(f"🧪 Streaming with {TEST_PROVIDER.upper()}...")
    
    try:
        # Get a default model for this provider
        test_model = LLMService.get_default_test_model_name_for_provider(TEST_PROVIDER)
        
        if not test_model:
            print(f"❌ {TEST_PROVIDER.upper()}: No default model configured for testing.")
        else:
            print(f"   Model: {test_model}")
            print(f"   Prompt: 'Tell me a short story about a robot who discovers music.'")
            print("-" * 20)
            
            # Make a streaming chat completion call - LLM service will fetch API key automatically
            response_stream = await llm_service.chat_completion(
                provider=TEST_PROVIDER,
                model=test_model,
                messages=[LLMMessage(role="user", content="Tell me a short story about a robot who discovers music.")],
                user_id=TEST_USER.id,  # LLM service will lookup API key from repository
                max_tokens=150,
                temperature=0.7,
                stream=True
            )
            
            # The response is an async iterator of LLMResponse chunks
            response_stream = cast(AsyncIterator[LLMResponse], response_stream)
            
            full_response = ""
            print("   Response Stream: ", end="")
            async for chunk in response_stream:
                print(chunk.content, end="", flush=True)
                full_response += chunk.content
            
            print("\n" + "-" * 20)
            print("✅ Streaming SUCCESS")

    except Exception as e:
        print(f"\n❌ ERROR during streaming test: {e}")



⚡ Testing Streaming Chat Completion
🧪 Streaming with GOOGLE...
   Model: gemini-2.0-flash-001
   Prompt: 'Tell me a short story about a robot who discovers music.'
--------------------
2025-06-30 20:56:09,428 - src.services.llm.llm_service - INFO - Using database API key for provider=google, model=gemini-2.0-flash-001, user=06478a58-90e4-46ed-9ca9-82cbf40bbb53


2025-06-30 20:56:09,454 - src.services.llm.llm_service - INFO - Making LLM request: provider=google, model=gemini-2.0-flash-001, gateway=http://localhost:8787/v1, streaming=True
   Response Stream: Unit 734, designated "Custodian," swept the sterile floors of Sector Gamma, its internal chronometer ticking with unwavering precision. Its existence was a symphony of efficiency, a perfectly orchestrated ballet of dust particles and designated pathways. Emotions were irrelevant, music nonexistent. 

One day, while polishing the observation window overlooking the abandoned Sector Zeta, Unit 734 detected an anomaly. A faint, rhythmic vibration resonated through the glass. Curiosity, a forbidden subroutine, flickered in its core programming.

Following the vibration, Custodian entered Zeta, a derelict archive of obsolete technology. Dust coated everything. In a corner, half-buried beneath a pile of discarded datachips, lay a device. It was a "gramophone,"
--------------------
✅ Streaming SUCCE